In [8]:
!nvidia-smi

Thu Nov 16 19:17:31 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [10]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git


  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-b_aoe6i1
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-b_aoe6i1
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4294 sha256=374c0bbe7afdd88224f8884f57d33e6022a23d4a9c7e184d836327f610d28344
  Stored in directory: /tmp/pip-ephem-wheel-cache-33f2f6r0/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


#Section Chapter 2

In [6]:
%load_ext nvcc_plugin


created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
#include <cuda.h>
#include <cuda_runtime.h>
#include <device_launch_parameters.h>
#include <iostream>

__global__ void vectorAddKernel(const float* A, const float* B, float* C, int n)
{
    int i = blockDim.x * blockIdx.x + threadIdx.x;
    if (i < n) {
        C[i] = A[i] + B[i];
    }
}

void vecAdd(float* A, float* B, float* C, int n)
{
    int size = n * sizeof(float);
    float *d_A, *d_B, *d_C;

    // Allocate device memory for A, B, and C
    cudaMalloc((void**)&d_A, size);
    cudaMalloc((void**)&d_B, size);
    cudaMalloc((void**)&d_C, size);

    // Copy A and B to device memory
    cudaMemcpy(d_A, A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, B, size, cudaMemcpyHostToDevice);

    // Define grid and block dimensions
    int threadsPerBlock = 256;  // You can adjust this based on your GPU
    int blocksPerGrid = (n + threadsPerBlock - 1) / threadsPerBlock;

    // Kernel launch code – to have the device perform the actual vector addition
    vectorAddKernel<<<blocksPerGrid, threadsPerBlock>>>(d_A, d_B, d_C, n);

    // Copy C from the device memory
    cudaMemcpy(C, d_C, size, cudaMemcpyDeviceToHost);

    // Free device vectors
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
}

int main()
{
    int n = 10;  // You can change the size of the vectors as needed
    float *A, *B, *C;

    A = (float*)malloc(n * sizeof(float));
    B = (float*)malloc(n * sizeof(float));
    C = (float*)malloc(n * sizeof(float));

    // Initialize A and B here

    // Inicializa los vectores A y B
    for (int i = 0; i < n; i++) {
        A[i] = static_cast<float>(i);  // Puedes cambiar estos valores
        B[i] = static_cast<float>(2 * i);
    }

    vecAdd(A, B, C, n);

    // Imprime los vectores A, B y el resultado C
    std::cout << "Vector A: ";
    for (int i = 0; i < n; i++) {
        std::cout << A[i] << " ";
    }
    std::cout << std::endl;

    std::cout << "Vector B: ";
    for (int i = 0; i < n; i++) {
        std::cout << B[i] << " ";
    }
    std::cout << std::endl;

    std::cout << "Resultado C: ";
    for (int i = 0; i < n; i++) {
        std::cout << C[i] << " ";
    }
    std::cout << std::endl;



    // C now contains the result of the vector addition

    // Don't forget to free host memory when you're done
    free(A);
    free(B);
    free(C);

    return 0;
}




Vector A: 0 1 2 3 4 5 6 7 8 9 
Vector B: 0 2 4 6 8 10 12 14 16 18 
Resultado C: 0 3 6 9 12 15 18 21 24 27 



In [ ]:
import tensorflow as tf
if tf.test.gpu_device_name():
    print('GPU encontrada:')
    print(tf.test.gpu_device_name())
else:
    print('No se encontró una GPU. Asegúrate de que has habilitado el entorno de ejecución de GPU en el menú "Entorno de ejecución".')


GPU encontrada:
/device:GPU:0


In [ ]:
%%cu

#include <cuda.h>
#include <cuda_runtime.h>
#include <device_launch_parameters.h>
#include <iostream>

//Tener en cuenta que instalamos lo necesario para correr codigo c en colab
//Por ello es necesario %%cu


//El kernel suma los elementos correspondientes de los vectores
__global__ void vectorAddKernel(const float* A, const float* B, float* C, int n)
{
    int i = blockDim.x * blockIdx.x + threadIdx.x;
    //garantizamos que cada thread proceso solo elementos validos del vetcor
    if (i < n) {
        C[i] = A[i] + B[i];
    }
}

void vecAdd(float* A, float* B, float* C, int n)
{
    int size = n * sizeof(float);
    float *d_A, *d_B, *d_C;

    // Registramos memoria para A, B, y C
    cudaMalloc((void**)&d_A, size);
    cudaMalloc((void**)&d_B, size);
    cudaMalloc((void**)&d_C, size);

    // Copy A and B to device memory
    cudaMemcpy(d_A, A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, B, size, cudaMemcpyHostToDevice);

    // Define grid and block dimensions
    int threadsPerBlock = 256;  // You can adjust this based on your GPU
    int blocksPerGrid = (n + threadsPerBlock - 1) / threadsPerBlock;

    // Kernel launch code – to have the device perform the actual vector addition
    vectorAddKernel<<<blocksPerGrid, threadsPerBlock>>>(d_A, d_B, d_C, n);

    // Copy C from the device memory
    cudaMemcpy(C, d_C, size, cudaMemcpyDeviceToHost);

    // Free device vectors
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
}

int main()
{
    //Agregamos el tamaño de vector para ver los cambios en la RAM de la GPU
    int n = 501020000;
    float *A, *B, *C;

    //
    A = (float*)malloc(n * sizeof(float));
    B = (float*)malloc(n * sizeof(float));
    C = (float*)malloc(n * sizeof(float));
h
    // Inicializa los vectores A y B
    // Valor del primer vector sera el doble de segundo vector
    for (int i = 0; i < n; i++) {
        A[i] = static_cast<float>(i);
        B[i] = static_cast<float>(2 * i);
    }

    vecAdd(A, B, C, n);

    // Imprime los vectores A, B y el resultado C
    std::cout << "Vector A: ";
    for (int i = 0; i < n; i++) {
        std::cout << A[i] << " ";
    }
    std::cout << std::endl;

    std::cout << "Vector B: ";
    for (int i = 0; i < n; i++) {
        std::cout << B[i] << " ";
    }
    std::cout << std::endl;

    std::cout << "Resultado C: ";
    for (int i = 0; i < n; i++) {
        std::cout << C[i] << " ";
    }
    std::cout << std::endl;



    // C now contains the result of the vector addition

    // Don't forget to free host memory when you're done
    free(A);
    free(B);
    free(C);

    return 0;
}




UsageError: Cell magic `%%cu` not found.


#Section Chapter 3

In [ ]:
!wget -o imagen.jpg https://www.24horas.cl/24horas/site/artic/20220910/imag/foto_0000000220220910153604/gatito.jpg

In [ ]:
!pip install opencv-python
!pip install opencv-python-headless
!apt-get -qq install -y libopencv-dev

E: Command line option '�' [from -y libopencv-dev] is not understood in combination with the other options.


In [7]:
%%writefile my_main.cu
#include <iostream>
#include <opencv2/opencv.hpp>

__global__ void colorToGreyscaleConversion(unsigned char *Pout, unsigned char *Pin, int width, int height) {
    int Col = threadIdx.x + blockIdx.x * blockDim.x;
    int Row = threadIdx.y + blockIdx.y * blockDim.y;
    if (Col < width && Row < height) {
        // get 1D coordinate for the grayscale image
        int grayOffset = Row * width + Col;
        // one can think of the RGB image having
        // CHANNEL times columns than the grayscale image
        int rgbOffset = grayOffset * 3; // Assuming 3 channels for RGB
        unsigned char r = Pin[rgbOffset];     // red value for pixel
        unsigned char g = Pin[rgbOffset + 1]; // green value for pixel
        unsigned char b = Pin[rgbOffset + 2]; // blue value for pixel
        // perform the rescaling and store it
        // We multiply by floating point constants
        Pout[grayOffset] = 0.21f * r + 0.71f * g + 0.07f * b;
    }
}

int main() {
    cv::Mat Imagen = cv::imread("/content/gatito.jpg");  // Replace with the actual path

    if (Imagen.empty()) {
        std::cerr << "Error: Unable to load the image." << std::endl;
        return -1;
    }

    const int width = Imagen.cols;
    const int height = Imagen.rows;
    const dim3 blockSize(16, 16);
    const dim3 gridSize((width + blockSize.x - 1) / blockSize.x, (height + blockSize.y - 1) / blockSize.y);

    unsigned char *d_input, *d_output;
    cudaMalloc(&d_input, width * height * 3 * sizeof(unsigned char));
    cudaMalloc(&d_output, width * height * sizeof(unsigned char));

    cudaMemcpy(d_input, Imagen.data, width * height * 3 * sizeof(unsigned char), cudaMemcpyHostToDevice);

    colorToGreyscaleConversion<<<gridSize, blockSize>>>(d_output, d_input, width, height);

    unsigned char *outputData = new unsigned char[width * height];
    cudaMemcpy(outputData, d_output, width * height * sizeof(unsigned char), cudaMemcpyDeviceToHost);

    cv::Mat outputImage(height, width, CV_8U, outputData);
    cv::imwrite("/content/gatitoblancoynegro.png", outputImage);
    cv::waitKey(0);


    cudaFree(d_input);
    cudaFree(d_output);

    return 0;
}

Writing my_main.cu


In [ ]:
!nvcc -o main my_main.cu -I/usr/include/opencv4 -L/usr/lib/x86_64-linux-gnu -lopencv_core -lopencv_highgui -lopencv_imgcodecs

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::buildMaps" is only partially overridden in class "cv::detail::AffineWarper"

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::warp" is only partially overridden in class "cv::detail::AffineWarper"

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(100): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is only partially overridden in class "cv::detail::FeatherBlender"

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(127): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is only partially overridden in class "cv::detail::MultiBandBlender"

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::buildMaps" is only partially ov

In [ ]:
!./main

In [ ]:
%%writefile my_main2.cu
#include <iostream>
#include <opencv2/opencv.hpp>

#define BLUR_SIZE 10

__global__ void blurkernel(unsigned char *in, unsigned char *out, int w, int h, int channels) {
    int Col = blockIdx.x * blockDim.x + threadIdx.x;
    int Row = blockIdx.y * blockDim.y + threadIdx.y;

    if (Col < w && Row < h) {
        int pixelIdx = (Row * w + Col) * channels;

        for (int k = 0; k < channels; k++) {
            int pixVal = 0;
            int pixels = 0;

            // Obtener el promedio de la caja circundante BLUR_SIZE × BLUR_SIZE
            for (int blurRow = -BLUR_SIZE; blurRow < BLUR_SIZE + 1; ++blurRow) {
                for (int blurCol = -BLUR_SIZE; blurCol < BLUR_SIZE + 1; ++blurCol) {
                    int curRow = Row + blurRow;
                    int curCol = Col + blurCol;

                    // Verificar que tengamos un píxel de imagen válido
                    if (curRow > -1 && curRow < h && curCol > -1 && curCol < w) {
                        int curPixelIdx = (curRow * w + curCol) * channels;
                        pixVal += in[curPixelIdx + k];
                        pixels++; // Realizar un seguimiento del número de píxeles en el promedio
                    }
                }
            }

            // Escribir nuestro nuevo valor de píxel
            out[pixelIdx + k] = (unsigned char)(pixVal / pixels);
        }
    }
}

int main() {
    cv::Mat Imagen = cv::imread("/content/gatito.jpg");  // Reemplaza con la ruta correcta

    if (Imagen.empty()) {
        std::cerr << "Error: No se puede cargar la imagen." << std::endl;
        return -1;
    }

    const int width = Imagen.cols;
    const int height = Imagen.rows;
    const dim3 blockSize(16, 16, 1);
    const dim3 gridSize((width + blockSize.x - 1) / blockSize.x, (height + blockSize.y - 1) / blockSize.y);

    unsigned char *d_input, *d_output;
    cudaMalloc(&d_input, width * height * 3 * sizeof(unsigned char));
    cudaMalloc(&d_output, width * height * 3 * sizeof(unsigned char));

    cudaMemcpy(d_input, Imagen.data, width * height * 3 * sizeof(unsigned char), cudaMemcpyHostToDevice);

    blurkernel<<<gridSize, blockSize>>>(d_input, d_output, width, height, 3);

    unsigned char *outputData = new unsigned char[width * height * 3];
    cudaMemcpy(outputData, d_output, width * height * 3 * sizeof(unsigned char), cudaMemcpyDeviceToHost);

    cv::Mat outputImage(height, width, CV_8UC3, outputData);
    cv::imwrite("/content/gatitoborroso.png", outputImage);

    cudaFree(d_input);
    cudaFree(d_output);

    return 0;
}

Overwriting my_main2.cu


In [ ]:
!nvcc -o main my_main2.cu -I/usr/include/opencv4 -L/usr/lib/x86_64-linux-gnu -lopencv_core -lopencv_highgui -lopencv_imgcodecs

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::buildMaps" is only partially overridden in class "cv::detail::AffineWarper"

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::warp" is only partially overridden in class "cv::detail::AffineWarper"

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(100): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is only partially overridden in class "cv::detail::FeatherBlender"

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(127): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is only partially overridden in class "cv::detail::MultiBandBlender"

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::buildMaps" is only partially ov

In [ ]:
!./main

#Section Chapter 4

In [44]:
%%writefile my_main.cu
#include <iostream>
#include <cuda_runtime.h>

// Kernel para la multiplicación de matrices
__global__ void MatrixMulKernel(float* M, float* N, float* P, int Width) {
    // Calcular el índice de fila del elemento P y M
    int Row = blockIdx.y * blockDim.y + threadIdx.y;
    // Calcular el índice de columna de P y N
    int Col = blockIdx.x * blockDim.x + threadIdx.x;

    if ((Row < Width) && (Col < Width)) {
        float Pvalue = 0;

        // Cada hilo calcula un elemento de la submatriz del bloque
        for (int k = 0; k < Width; ++k) {
            Pvalue += M[Row * Width + k] * N[k * Width + Col];
        }

        P[Row * Width + Col] = Pvalue;
    }
}

int main() {
    // Definir el tamaño de las matrices
    const int Width = 256;

    // Inicializar matrices de entrada y salida en la CPU
    float hostM[Width * Width];
    float hostN[Width * Width];
    float hostP[Width * Width];

    // Inicializar matrices con datos de ejemplo
    for (int i = 0; i < Width * Width; ++i) {
        hostM[i] = i;
        hostN[i] = i * 2;
    }

    // Copiar matrices de la CPU a la GPU
    float *deviceM, *deviceN, *deviceP;
    cudaMalloc((void**)&deviceM, Width * Width * sizeof(float));
    cudaMalloc((void**)&deviceN, Width * Width * sizeof(float));
    cudaMalloc((void**)&deviceP, Width * Width * sizeof(float));

    cudaMemcpy(deviceM, hostM, Width * Width * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(deviceN, hostN, Width * Width * sizeof(float), cudaMemcpyHostToDevice);

    // Configurar la cuadrícula y los bloques de hilos
    dim3 dimGrid(1, 1, 1);
    dim3 dimBlock(Width, Width, 1);

    // Medir el tiempo de compilación
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Iniciar el temporizador
    cudaEventRecord(start);

    // Llamar al kernel de multiplicación de matrices
    MatrixMulKernel<<<dimGrid, dimBlock>>>(deviceM, deviceN, deviceP, Width);

    // Detener el temporizador
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);

    std::cout << "Tiempo de compilación del kernel: " << milliseconds << " ms" << std::endl;

    // Copiar la matriz de salida desde la GPU a la CPU
    cudaMemcpy(hostP, deviceP, Width * Width * sizeof(float), cudaMemcpyDeviceToHost);

    // Imprimir la matriz de salida
    /*
    std::cout << "Matriz de salida (P):" << std::endl;
    for (int i = 0; i < Width; ++i) {
        for (int j = 0; j < Width; ++j) {
            std::cout << hostP[i * Width + j] << " ";
        }
        std::cout << std::endl;
    }*/

    // Liberar memoria en la GPU
    cudaFree(deviceM);
    cudaFree(deviceN);
    cudaFree(deviceP);

    return 0;
}



Overwriting my_main.cu


In [45]:
!nvcc -o my_main my_main.cu

In [46]:
!./my_main

Tiempo de compilación del kernel: 0.00288 ms


In [107]:
%%writefile my_main.cu
#include <iostream>
#include <cstdlib>
#include <ctime>

// Define TILE_WIDTH
#define TILE_WIDTH 32 // Tamaño del bloque de hilo (puedes ajustarlo según sea necesario)

// Kernel de multiplicación de matrices con memoria compartida
__global__ void MatrixMulKernel(float* d_M, float* d_N, float* d_P, int Width) {
    __shared__ float Mds[TILE_WIDTH][TILE_WIDTH];
    __shared__ float Nds[TILE_WIDTH][TILE_WIDTH];
    int bx = blockIdx.x;
    int by = blockIdx.y;
    int tx = threadIdx.x;
    int ty = threadIdx.y;
    int Row = by * TILE_WIDTH + ty;
    int Col = bx * TILE_WIDTH + tx;
    float Pvalue = 0;

    for (int ph = 0; ph < Width / TILE_WIDTH; ++ph) {
        Mds[ty][tx] = d_M[Row * Width + ph * TILE_WIDTH + tx];
        Nds[ty][tx] = d_N[(ph * TILE_WIDTH + ty) * Width + Col];
        __syncthreads();

        for (int k = 0; k < TILE_WIDTH; ++k) {
            Pvalue += Mds[ty][k] * Nds[k][tx];
        }

        __syncthreads();
    }

    d_P[Row * Width + Col] = Pvalue;
}


// Function to initialize matrices with random values
void initializeMatrix(float* matrix, int size) {
    for (int i = 0; i < size; ++i) {
        matrix[i] = static_cast<float>(rand()) / RAND_MAX; // Random values between 0 and 1
    }
}

// Function to print a matrix
void printMatrix(const float* matrix, int rows, int cols) {
    for (int i = 0; i < rows; ++i) {
        for (int j = 0; j < cols; ++j) {
            std::cout << matrix[i * cols + j] << " ";
        }
        std::cout << std::endl;
    }
}

int main() {
    // Matrix dimensions
    const int Width = 2048;
    const int MatrixSize = Width * Width;

    // Host matrices
    float *h_M = new float[MatrixSize];
    float *h_N = new float[MatrixSize];
    float *h_P = new float[MatrixSize];

    // Initialize matrices
    initializeMatrix(h_M, MatrixSize);
    initializeMatrix(h_N, MatrixSize);

    // Device matrices
    float *d_M, *d_N, *d_P;
    cudaMalloc((void**)&d_M, MatrixSize * sizeof(float));
    cudaMalloc((void**)&d_N, MatrixSize * sizeof(float));
    cudaMalloc((void**)&d_P, MatrixSize * sizeof(float));

    // Copy matrices from host to device
    cudaMemcpy(d_M, h_M, MatrixSize * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_N, h_N, MatrixSize * sizeof(float), cudaMemcpyHostToDevice);

    // Configure grid and block dimensions
    dim3 dimGrid(Width / TILE_WIDTH, Width / TILE_WIDTH, 1);
    dim3 dimBlock(TILE_WIDTH, TILE_WIDTH, 1);

    // Measure execution time
    clock_t start_time = clock();

    // Call the kernel
    MatrixMulKernel<<<dimGrid, dimBlock>>>(d_M, d_N, d_P, Width);

    // Synchronize to wait for the kernel to finish
    cudaDeviceSynchronize();

    // Measure execution time
    clock_t end_time = clock();
    double elapsed_time = static_cast<double>(end_time - start_time) / CLOCKS_PER_SEC;

    std::cout << "Matrix multiplication took " << elapsed_time << " seconds." << std::endl;

    // Copy the result back from the device to the host
    cudaMemcpy(h_P, d_P, MatrixSize * sizeof(float), cudaMemcpyDeviceToHost);

    // Print the result (if the matrix is small)
    if (Width <= 16) {
        std::cout << "Result Matrix:" << std::endl;
        printMatrix(h_P, Width, Width);
    }

    // Free allocated memory
    delete[] h_M;
    delete[] h_N;
    delete[] h_P;
    cudaFree(d_M);
    cudaFree(d_N);
    cudaFree(d_P);

    return 0;
}



Overwriting my_main.cu


In [108]:
!nvcc -o my_main my_main.cu

In [109]:
!./my_main

Matrix multiplication took 0.04169 seconds.
